# Log Analysis Tool: Hacker's Diary Investigation
This notebook has been updated to analyze system access logs for suspicious login behavior, detect brute-force patterns, recover leaked credentials.

In [1]:
# Import Required Libraries
import tkinter as tk
from tkinter import filedialog
from tkinter.scrolledtext import ScrolledText
import re
from collections import Counter

In [2]:
# Define Function to Open and Read Log File
def open_log_file():
    file_path = filedialog.askopenfilename(filetypes=[("Log files", "*.log"), ("All files", "*.*")])
    if file_path:
        with open(file_path, 'r') as file:
            log_text.delete("1.0", tk.END)
            log_text.insert(tk.END, file.read())

In [3]:
# Define Function to Extract and Display Log Details
def analyze_logs():
    log_content = log_text.get("1.0", tk.END)
    ip_pattern = r"\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b"
    request_pattern = r'"(GET|POST|PUT|DELETE)'
    status_pattern = r'\s(\d{3})\s'

    ips = re.findall(ip_pattern, log_content)
    requests = re.findall(request_pattern, log_content)
    statuses = re.findall(status_pattern, log_content)

    ip_count = Counter(ips)
    request_count = Counter(requests)
    status_count = Counter(statuses)

    result_text.delete("1.0", tk.END)
    result_text.insert(tk.END, "IP Address Counts:\n")
    for ip, count in ip_count.items():
        result_text.insert(tk.END, f"{ip}: {count}\n")

    result_text.insert(tk.END, "\nRequest Type Counts:\n")
    for req, count in request_count.items():
        result_text.insert(tk.END, f"{req}: {count}\n")

    result_text.insert(tk.END, "\nStatus Code Counts:\n")
    for status, count in status_count.items():
        result_text.insert(tk.END, f"{status}: {count}\n")

In [4]:
# Define Function to Analyze Logs for Suspicious Behavior
def analyze_logs_for_suspicious_activity(log_content):
    failed_login_pattern = r"Failed login attempt from (\d+\.\d+\.\d+\.\d+) - User: (\w+) - Password: (\w+)"
    successful_login_pattern = r"Successful login from (\d+\.\d+\.\d+\.\d+) - User: (\w+) - Password: (\w+)"
    flag_pattern = r"FLAG\{.*?\}"

    failed_logins = re.findall(failed_login_pattern, log_content)
    successful_logins = re.findall(successful_login_pattern, log_content)
    flags = re.findall(flag_pattern, log_content)

    # Count failed login attempts by IP
    failed_login_ips = Counter([entry[0] for entry in failed_logins])

    # Display Results
    result_text.insert(tk.END, "Suspicious Login Behavior Detected:\n")
    result_text.insert(tk.END, "\nFailed Login Attempts by IP:\n")
    for ip, count in failed_login_ips.items():
        result_text.insert(tk.END, f"{ip}: {count} attempts\n")

    result_text.insert(tk.END, "\nRecovered Credentials from Successful Logins:\n")
    for entry in successful_logins:
        result_text.insert(tk.END, f"IP: {entry[0]}, User: {entry[1]}, Password: {entry[2]}\n")

    if flags:
        result_text.insert(tk.END, "\nHidden Flags Found:\n")
        for flag in flags:
            result_text.insert(tk.END, f"{flag}\n")
    else:
        result_text.insert(tk.END, "\nNo Hidden Flags Found.\n")

In [ ]:
# Create GUI Application
root = tk.Tk()
root.title("Log Analysis Tool")

frame = tk.Frame(root)
frame.pack(pady=10)

open_button = tk.Button(frame, text="Open Log File", command=open_log_file)
open_button.pack(side=tk.LEFT, padx=5)

analyze_button = tk.Button(frame, text="Analyze Logs", command=analyze_logs)
analyze_button.pack(side=tk.LEFT, padx=5)

suspicious_button = tk.Button(frame, text="Analyze Suspicious Activity", command=lambda: analyze_logs_for_suspicious_activity(log_text.get("1.0", tk.END)))
suspicious_button.pack(side=tk.LEFT, padx=5)

log_text = ScrolledText(root, width=80, height=20, wrap=tk.WORD)
log_text.pack(padx=10, pady=10)

result_text = ScrolledText(root, width=80, height=20, wrap=tk.WORD)
result_text.pack(padx=10, pady=10)

root.mainloop()